In [52]:
import pandas as pd
import glob
import os
import datetime as dt
import altair as alt
import numpy as np

In [20]:
# join csvs
files = os.path.join('./archive', '*.csv')
files = glob.glob(files)

# joining files with concat and read_csv
df = pd.concat(map(pd.read_csv, files), ignore_index=True)
df.head()

df['High - Low % change'] = ((df['High'] - df['Low']) / df['Low']) * 100

In [21]:
df.head()

,SNo,Name,Symbol,Date,High,Low,Open,Close,Volume,Marketcap,High - Low % change
0,1,NEM,XEM,2015-04-02 23:59:59,0.000323,0.000227,0.000242,0.000314,28549.400391,2.823534e+06,42.151030
1,2,NEM,XEM,2015-04-03 23:59:59,0.000330,0.000291,0.000309,0.000310,20677.900391,2.792457e+06,13.379166
2,3,NEM,XEM,2015-04-04 23:59:59,0.000318,0.000251,0.000310,0.000277,23550.199219,2.488770e+06,27.033231
3,4,NEM,XEM,2015-04-05 23:59:59,0.000283,0.000218,0.000272,0.000232,26800.199219,2.087388e+06,29.490301
4,5,NEM,XEM,2015-04-06 23:59:59,0.000299,0.000229,0.000232,0.000289,22511.500000,2.598354e+06,30.219999


In [22]:
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,SNo,Name,Symbol,Date,High,Low,Open,Close,Volume,Marketcap,High - Low % change
0,1,NEM,XEM,2015-04-02 23:59:59,0.000323,0.000227,0.000242,0.000314,28549.400391,2.823534e+06,42.151030
1,2,NEM,XEM,2015-04-03 23:59:59,0.000330,0.000291,0.000309,0.000310,20677.900391,2.792457e+06,13.379166
2,3,NEM,XEM,2015-04-04 23:59:59,0.000318,0.000251,0.000310,0.000277,23550.199219,2.488770e+06,27.033231
3,4,NEM,XEM,2015-04-05 23:59:59,0.000283,0.000218,0.000272,0.000232,26800.199219,2.087388e+06,29.490301
4,5,NEM,XEM,2015-04-06 23:59:59,0.000299,0.000229,0.000232,0.000289,22511.500000,2.598354e+06,30.219999


In [23]:
range_max = df['Date'].max()
range_min = range_max - dt.timedelta(days=30)

sliced_df = df[(df['Date'] >= range_min) & 
               (df['Date'] <= range_max)]
sliced_df.head()

,SNo,Name,Symbol,Date,High,Low,Open,Close,Volume,Marketcap,High - Low % change
2257,2258,NEM,XEM,2021-06-06 23:59:59,0.190063,0.179704,0.180779,0.187718,84255611.92,1.689458e+09,5.764352
2258,2259,NEM,XEM,2021-06-07 23:59:59,0.192372,0.170055,0.187720,0.170997,87266392.14,1.538975e+09,13.122874
2259,2260,NEM,XEM,2021-06-08 23:59:59,0.173177,0.153965,0.171363,0.165499,94018710.02,1.489493e+09,12.478176
2260,2261,NEM,XEM,2021-06-09 23:59:59,0.175315,0.158113,0.164760,0.173861,98169054.17,1.564746e+09,10.879963
2261,2262,NEM,XEM,2021-06-10 23:59:59,0.177479,0.166559,0.174331,0.170198,79409647.01,1.531782e+09,6.556448


In [24]:
sliced_df_2 = sliced_df[(sliced_df['Name'] == "Bitcoin")
                    | (sliced_df['Name'] == "Dogecoin")
                    | (sliced_df['Name'] == "Ethereum")
                    | (sliced_df['Name'] == "Litecoin")
                    | (sliced_df['Name'] == "Binance Coin")
                    | (sliced_df['Name'] == "Tether")]

In [81]:
# todo: figure out how to get symbol from hovering over
pct_change_lineplot = alt.Chart(sliced_df_2).mark_line().encode(
    x=alt.X('Date:T', axis = alt.Axis(title = 'Date'.upper(), format = ("%m/%d %H:%M"))),
    y='High - Low % change:Q',
    color='Symbol:N',
    strokeDash='Symbol:N',
    tooltip=[alt.Tooltip('Symbol', title='Symbol', format='.0%')]
    ).interactive()

In [82]:
pct_change_lineplot

alt.Chart(...)

In [84]:
pct_change_lineplot.save('pct_change_lineplot.json')

In [26]:
# TODO: find avalanche?
sliced_df_3 = sliced_df[(sliced_df['Name'] == "Bitcoin")
                    | (sliced_df['Name'] == "Tezos")
                    | (sliced_df['Name'] == "Ethereum")
                    | (sliced_df['Name'] == "Litecoin")
                    | (sliced_df['Symbol'] == "AVAX")]

In [48]:
# multiline tooltip source: https://altair-viz.github.io/gallery/multiline_tooltip.html

# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['Date'], empty='none')

line = alt.Chart(sliced_df_3).mark_line().encode(
    x=alt.X('Date:T', axis = alt.Axis(title = 'Date'.upper(), format = ("%m/%d %H:%M"))),
    y='High - Low % change:Q',
    color='Symbol:N',
    strokeDash='Symbol:N' 
    ) 

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(sliced_df_3).mark_point().encode(
    x='Date:T',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='center', dx=0, dy=-5).encode(
    text=alt.condition(nearest, 'High - Low % change:Q', alt.value(' ')),
)

# Draw a rule at the location of the selection
rules = alt.Chart(sliced_df_3).mark_rule(color='gray').encode(
    x='Date:T',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
final = alt.layer(
    line, selectors, points, rules, text
).properties(
    width=600, height=300
).interactive()

In [49]:
final.save('tooltipvis.json')

In [46]:
final

alt.LayerChart(...)

In [71]:
# TODO: plot standard deviations
# df_3 = df.groupby('Symbol', as_index=False)['High - Low % change', 'Close'].agg(np.std, ddof=1)
df_3 = df.groupby('Symbol', as_index=False)['High - Low % change'].std(ddof=1)
df_4 = df.groupby('Symbol', as_index=False)['Volume'].mean()
df_5 = pd.merge(df_3, df_4, on=['Symbol'])

print(df_5)

   Symbol  High - Low % change        Volume
0    AAVE             9.569807  5.023421e+08
1     ADA            10.179917  8.934183e+08
2    ATOM             9.311348  2.963413e+08
3     BNB            11.104655  6.269804e+08
4     BTC             5.337787  1.090633e+10
5     CRO            11.915715  5.003678e+07
6    DOGE            17.855694  4.326321e+08
7     DOT            11.095654  2.006507e+09
8     EOS            11.224559  1.888406e+09
9     ETH             9.709850  7.057058e+09
10   LINK             9.845097  6.923608e+08
11    LTC             8.866763  1.284851e+09
12  MIOTA             9.655488  5.672587e+07
13    SOL            10.709899  1.956751e+08
14    TRX            48.035015  9.459435e+08
15    UNI            13.624274  1.061551e+09
16   USDC             1.222096  6.513472e+08
17   USDT             1.582586  1.938484e+10
18   WBTC            14.277258  5.547932e+07
19    XEM            10.693610  4.428130e+07
20    XLM            12.373240  2.515161e+08
21    XMR 

In [78]:
std_chart = alt.Chart(df_5).mark_circle(size=60).encode(
    x='Volume:Q',
    y=alt.Y('High - Low % change', axis=alt.Axis(format='$', title='High-Low % Change Standard Deviation')),
    color='Symbol',
    tooltip=['Symbol', alt.Tooltip('High - Low % change', title='Standard Deviation'), 'Volume']
).interactive()

In [79]:
std_chart

alt.Chart(...)

In [80]:
std_chart.save('std_chart.json')